### Make sure that the libraries you use are implementing standard hash functions 
#### Hashing libraries may be implemented in varous programming languages and for various operating systems. The libraries must implement algorithms in consistent ways defined by national and international standard bodies. For example:

- SHA-2 family of hash algorithms: SHA-224, SHA-256, SHA-384, SHA-512, SHA-512/224, and SHA-512/256 implementations must comply with the [NIST FIPS 180-4](https://csrc.nist.gov/publications/detail/fips/180/4/final) standard. 
- SHA-3 family: SHA3-224, SHA3-256, SHA3-384, and SHA3-512 implementations must comply with [NIST FIPS 202](https://csrc.nist.gov/pubs/fips/202/final) standard.
- Implementations of [Argon2](https://en.wikipedia.org/wiki/Argon2) family of memory-hard hash functions must comply with [IETF's RFC 9106](https://datatracker.ietf.org/doc/html/rfc9106).  
- MD5 implementations must comply with [IETF's RFC 1321](https://www.ietf.org/rfc/rfc1321.txt). 

### Make sure that the hash algorithms you use are current! 
- SHA-1 is considered [not strong enough](https://www.nist.gov/news-events/news/2022/12/nist-retires-sha-1-cryptographic-algorithm) and is [recommended for replacement](https://csrc.nist.gov/news/2022/nist-transitioning-away-from-sha-1-for-all-apps) by either SHA-2 or SHA-3 families of algorithms.
- There are known computational optimizations for Argon2i and Argon2d hash functions, though Argon2id is recommended by [OWASP guidance](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html).
 


A bcrypt hash value consists of the following parts:

- \$2\$ (note that it is surrounded by "$" signs) is a usual bcrypt hash function prefix.
- \$2b\$ in our case informs that the hash is generated according to the OpenBSD implementation of bcrypt.
- besides 2b, other possible prefixes bcrypt can have are: 2a, 2x, and 2y.
- 12 — this is the "cost" parameter, indicating that the password is hashed with 210 (i.e. 1024) iterations of the blowfish cipher. A higher cost parameter results in password hashes that are harder to crack by brute force.
- yi32JXsMsobDwrNgtylgv. — This is the 128-bit salt value, encoded in base64 as 22 ASCII characters.

The remaining characters correspond to the 184-bit blowfish hash value, encoded in base64 as 31 characters.

Below is an illustration of using bcrypt. Use of Bcrypt can increase credential hashing time, if so desired. This is useful for mitigating brute-force attacks.  
If Bcrypt is not installed, you can add py-bcrypt to your default Python environment as follows:  

```
pip install py-bcrypt
```
or

```
python3 -m pip install py-bcrypt
```

In [13]:
# Demonstrating what bcrypt hash functions turn passwords into
import bcrypt   # !!! The Older Legacy Tool.
from datetime import datetime
from hashlib import pbkdf2_hmac

def hash_bcrypt(password: str, salt: bytes) -> str:
    pwd_bytes = password.encode('utf-8')
    pwd_hash = bcrypt.hashpw(pwd_bytes, salt)
    print(f'Bcrypt:\n\tPassword:\t{password}\n\tPwd_Bytes:\t{pwd_bytes}\n\tPwd_Salt:\t{salt}\n\tPwd_Hash:\t{pwd_hash}')
    return pwd_hash

# salt = bcrypt.gensalt()
salt = b'$2b$12$yi32JXsMsobDwrNgtylgv.'
p1 = 'password123'
p2 = 'password123🌺🌸'
p3 = '1234567890'

hash_bcrypt(p1,salt)
hash_bcrypt(p2,salt)
hash_bcrypt(p3,salt)

Bcrypt:
	Password:	password123
	Pwd_Bytes:	b'password123'
	Pwd_Salt:	b'$2b$12$yi32JXsMsobDwrNgtylgv.'
	Pwd_Hash:	b'$2b$12$yi32JXsMsobDwrNgtylgv.03APX9K7LbQ96.St8VHhKgz4xyX2kem'
Bcrypt:
	Password:	password123🌺🌸
	Pwd_Bytes:	b'password123\xf0\x9f\x8c\xba\xf0\x9f\x8c\xb8'
	Pwd_Salt:	b'$2b$12$yi32JXsMsobDwrNgtylgv.'
	Pwd_Hash:	b'$2b$12$yi32JXsMsobDwrNgtylgv.qhfrgptutkThfszppajtS5HS1YunxXm'
Bcrypt:
	Password:	1234567890
	Pwd_Bytes:	b'1234567890'
	Pwd_Salt:	b'$2b$12$yi32JXsMsobDwrNgtylgv.'
	Pwd_Hash:	b'$2b$12$yi32JXsMsobDwrNgtylgv.fuKDH5LhaYQ8s2g3twD4OI29iHHSgzG'


b'$2b$12$yi32JXsMsobDwrNgtylgv.fuKDH5LhaYQ8s2g3twD4OI29iHHSgzG'

#### The SHA Family of algorithms is implemented in the module ```hashlib```. The module ```hashlib``` is usually included in the default Python distributions on most platforms. For password hashing, SHA2 or SHA3 hash algorithms are plugged into the PBKDF2 HMAC hash chaining function, which is intentionally slowing down computation of the password hash by performing multiple iterations. For PBKDF2 hashing of the passwords NIST recommends the following:

- Generate and store a salt that is at least 128 bytes long.
- Use at least 100,000 hash chaining iterations.
- Allow maximum password length to be at least 64 characters.
- Do not constrain characters and allow UNICODE characters in the passwords.


In [29]:
import os
import hashlib



def do_and_print_hash(hash_algorithm, iterations_count, dk_length, p_word2, salt_value):
    the_hash_to_use = hashlib.pbkdf2_hmac(
                    hash_name=hash_algorithm, 
                    password = p_word2.encode('utf-8'),
                    salt = salt_value,
                    iterations=iterations_count, 
                    dklen = dk_length 
                    )

    print(f'\n\npbkdf2_hmac Algorithm with parameters:')
    print(f'\tHash-Name: {hash_algorithm}')
    print(f'\tRandom Salt[B]: {salt_value}') # .decode("utf-8")
    print(f'\tRandom Salt[H]: {salt_value.hex()}') # .decode("utf-8")
    print(f'\tIterations: {iterations_count}')
    print(f'\tDesired Length: {dk_length}')
    print(f'\tPassword: {p_word2} ')
    print(f'\tResult Hash[B]: {the_hash_to_use}')
    print(f'\tResult Hash[H]: {the_hash_to_use.hex()}')
    return the_hash_to_use

# The parameters that are determining the HASHING
salt_length = 128 # <= This is NIST recommended salt size 
# hash_algorithm = 'sha256'

hash_algorithm = 'sha512'
iterations_count = 100000
dk_length = 512

p_word1 = 'password123'
p_word2 = 'password123🌺🌸'
p_word3 = '1234567890'

# salt_value1 = os.urandom(salt_length) # <= Random salt
# Frozen Salt for consistency
salt_value1 = b'\xe6\xf0R\x8c\xc5\xb5\xa5\xa7\xbb\xff\xffu;V\x130\xfc\xc5\x10\xa7\x9c\x90\xb6W\xc8K\xa1\xd4{ ,\x16\xbe*1j\xe5\xf51/b\x19\x81\xbdma\xd4\x19\x1bq\x84e\xa2z\n\xe5\x07\x1d\tf\xbc\x0f\xcb\x82cBO\x947\t\x00O\t\x08\x1b\xe0\xa8\xc4\x0e\xc6\xb2\xc0\xcc,\x17\xec\xf0\x1d\xce\x8c=\xff\x8b\xdbE\x1c\xbbr\x05\xd6\xaa\xa0\x13s\xfe:c\x1e\xd0%\xb9\x81xi:\xbf\xd7\xe5\xb4\x14\xdfU\xc4\xd5{;\x10Z'
# salt_value2 = os.urandom(salt_length)
salt_value2 = b'\xd2\xa0\xc9*\x88\tt\xbf\xa1\x16\x80+J\r\xfan\x0c\x0c#i\xf8i\xa89\xb5\x90\xaeL\xe4K.\x17}z\xd7\x85\xe1C^\xbc\xa4\x99\xc2\xcd\xd0]\xe0\xa9`\x15\xa1@\xa6\x95T\xac\xcf\xda\xb9\xa9/\xc3_n\xf3\x11=\xfa\xfa\x8b\xb8\xb4\xad\xb6e\xbe\xbf\xb6\x10\xf5\\2K\x82%\x08\xd3\x15\xadE?\x1b\x17C\xde\xa1\xe0\x15\x8a\xac\xb3P!\xd4\xca\xd3\x90\xfc\xa6\xaf\xc8\x9aY\xa4\x7f\xc0:gp2\xf3\x0e\xd1&\x0b\x80\xd6Z'
# salt_value3 = os.urandom(salt_length)
salt_value3 =b'X\x82\xea\xcb\x8e;\xba\xd2\xd4\x9d\xfb\xc1T\xca\xb4\xe7\x93\x1b\xa0\xb6\x0f\xaa\xce\x13D\xf9\x19\xff+/\xe9]1\xaf)8\xdc\xc0\xe2\x06N\xf4@\x9e\rk\xe6\xd3Z \xceD6>"?R\xb7~\x8a|\xa5y%\x18\xbd\xec\xcc\xfc\t\xb0\xf9Oi\x0e\xe9f\x8e\xff\xf6\x9d\x90\x00\x03\xa3\x0f\x80[Q\xech\x9f\x07Dal\x89\xa3\\\x93K\xc1\xca\xbd\x1b\xa3\xa7\x04\x10\xbd\x10p\xe1\x1d\xa5\x1b\xf4\xb3\x95O\x93\x163\x89/\x9f\xf1\x16'

the_hash_to_use = do_and_print_hash(hash_algorithm, iterations_count, dk_length, p_word1, salt_value1)
the_hash_to_use = do_and_print_hash(hash_algorithm, iterations_count, dk_length, p_word2, salt_value2)
the_hash_to_use = do_and_print_hash(hash_algorithm, iterations_count, dk_length, p_word3, salt_value3)

# to transform Hex'' -> Bytes'' bytearray.fromhex('81ca0cc39f1cc7e8ae6caf5e4630d127')
# to transform Bytes'' -> Hex'' 'b'\x81\xca\x0c\xc3\x9f\x1c\xc7\xe8\xael\xaf^F0\xd1\'\xc2\x14\r\x9d\xbf\xce\xf4\xab\x8am\xd0\x8bLP'.hex()



pbkdf2_hmac Algorithm with parameters:
	Hash-Name: sha512
	Random Salt[B]: b'\xe6\xf0R\x8c\xc5\xb5\xa5\xa7\xbb\xff\xffu;V\x130\xfc\xc5\x10\xa7\x9c\x90\xb6W\xc8K\xa1\xd4{ ,\x16\xbe*1j\xe5\xf51/b\x19\x81\xbdma\xd4\x19\x1bq\x84e\xa2z\n\xe5\x07\x1d\tf\xbc\x0f\xcb\x82cBO\x947\t\x00O\t\x08\x1b\xe0\xa8\xc4\x0e\xc6\xb2\xc0\xcc,\x17\xec\xf0\x1d\xce\x8c=\xff\x8b\xdbE\x1c\xbbr\x05\xd6\xaa\xa0\x13s\xfe:c\x1e\xd0%\xb9\x81xi:\xbf\xd7\xe5\xb4\x14\xdfU\xc4\xd5{;\x10Z'
	Random Salt[H]: e6f0528cc5b5a5a7bbffff753b561330fcc510a79c90b657c84ba1d47b202c16be2a316ae5f5312f621981bd6d61d4191b718465a27a0ae5071d0966bc0fcb8263424f943709004f09081be0a8c40ec6b2c0cc2c17ecf01dce8c3dff8bdb451cbb7205d6aaa01373fe3a631ed025b98178693abfd7e5b414df55c4d57b3b105a
	Iterations: 100000
	Desired Length: 512
	Password: password123 
	Result Hash[B]: b'\xcb\xb8^\x16h\xdd\x961\xd8%55\x86\xbe\x83\xdf\xc2\xd2\x8a\xed\xac\x12\xb8SU\x8f\xcf\x1c\x19\xac\xa9q\x1d!1+e\xef\xf9\x12\xc1X\xa5\xa19\xad;\x83>$z"\x0fy\x1d\x0b\xb17[\xe1n\xb9\xd2R\x


## Here is a summary of [OWASP credential hashing guidance ](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html):

- For **legacy** systems using bcrypt, use a work factor of 10 or more and with a password limit of 72 bytes.
- Use Argon2id with a minimum configuration of 19 MiB of memory, an iteration count of 2, and 1 degree of parallelism.
- If Argon2id is not available on your platform, - use scrypt with a minimum CPU/memory cost parameter of (2^17), a minimum block size of 8 (1024 bytes), and a parallelization parameter of 1.
- If FIPS-140 compliance is required, use PBKDF2 with a work factor (iterations) of 600,000 or more and set with an internal hash function of HMAC  SHA-256.
- Consider combining a peppering implementation with additional defenses (but, if it is used alone, it provides no additional security characteristics).
